# Cascading Analysis and Synthesis PFBs
This example uses an ADC-DAC pair. The ADC feeds an Analysis Polyphase Filter Bank, which is cascaded with a Synthesis Polyphase Filter Bank with its output connected to the DAC. There is a block between both PFBs which is a simple channel mask. When the corresponding bit is set to logic 1, the channel on the Analysis PFB connects with the channel on the Synthesis PFB. When the bit is set to logic 0, the channel is zeroed-out and the corresponding band is eliminated on the overall response.

To run this example use the following configuration:
* Input : ADC_C
* Output: DAC_A

Ideally, you should be using an external VNA instrument or similar in order to get the frequency response of the filter. Output power on the VNA should be set to -30 dBm as it may saturate with higher power.

### Mixer and decimation/interpolation on converters
The internal fine-frequency mixer block is used on both ADC and DAC. The ADC applies a FIR low-pass followed by 4 times decimation. The frequency of the mixer can be programmed from the provided Python driver.
The DAC has a similar architecture, with 4 times interpolation followed by low-pass FIR filtering. This goes then into the mixer. Again, the mixer frequency can be controlled from the provided function.
A convenient class called FilterChain is provided with most of the functionality that is needed to use the filter. This also includes a convenient set_mixer_frequency function that sets both ADC and DAC mixer frequencies. It also takes care of mirroring, by applying a positive fmix on the DAC and negative fmix on the ADC to avoid recovering the opposite side-band.

### Filter programming
The filter consists of a simple bit mask that either connects or not one channel from the Analysis PFB into the corresponding channel of the Synthesis PFB. Convenient functions are provided to enable all channels, disable all channels, or define frequency bands for the filter that will internally map to specific PFB channels and set the corresponding bits.

In [17]:
import sys
sys.path.append('./soft')

from pfbs import *

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift

In [18]:
# Initialize Firmware.
soc = TopSoc('./pfbs_v1.bit')

# Print information.
print(soc)


QICK configuration:

	Board: RFSoC4x2

	Dual Chains
	Dual 0: 
		ADC: 224_0, fs = 4915.2 MHz, Decimation    = 4
		DAC: 228_0, fs = 4915.2 MHz, Interpolation = 4
		PFB: fs = 2.4 MHz, fc = 1.2 MHz, 1024 channels

	Filter Chains
	Filter 0: 
		ADC: 224_1, fs = 4915.2 MHz, Decimation    = 4
		DAC: 230_0, fs = 4915.2 MHz, Interpolation = 4
		PFB: fs = 9.6 MHz, fc = 4.8 MHz, 256 channels


In [19]:
###############################
### Initialize Filter Chain ###
###############################

# Filter chain: includes both analysis and synthesis, cascaded.
filt = FilterChain(soc, chain=soc['filter'][0])

In [20]:
############################
### Filter Configuration ###
############################
# Set quantization.
filt.analysis.qout(2)
filt.synthesis.qout(1)

# Set mixer.
filt.set_mixer_frequency(1000)

# Enable all channels.
filt.allon()

# Set filter band.
#filt.band(flow = 600, fhigh = 700, single=True)
#filt.band(flow = 900, fhigh = 1050, single=False)
#filt.band(flow = 1200, fhigh=1210, single=False)